# Reddit Comments from May 2015

This dataset contains every publicly available Reddit comment for the month of May, 2015. Approximately 37 million comments were made by 2.7 million unique authors, stored in a ~15 GB compressed file.

In [1]:
from spark.dataset import Dataset


hdfs_path = "hdfs://namenode:9000/user/spark/reddit_comments/"
spark_master = "spark://spark-master:7077"

dataset = Dataset(hdfs_path, spark_master)
dataset.pre_process()

# Test
dataset.dataframe.sort("score", ascending=False).show(10)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/01 18:59:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----------+------------+---------+----------+------------+----------------------+-----------------+-------+--------------+------+--------+------------------+-----+------------+--------------------+-------------+----------+----------------+----------+
|created_utc|subreddit_id|  link_id|      name|score_hidden|author_flair_css_class|author_flair_text|     id|removal_reason|gilded|archived|            author|score|retrieved_on|                body|distinguished|    edited|controversiality| parent_id|
+-----------+------------+---------+----------+------------+----------------------+-----------------+-------+--------------+------+--------+------------------+-----+------------+--------------------+-------------+----------+----------------+----------+
| 1431317199|    t5_2qh1i|t3_35jfjt|t1_cr56nez|           0|                  NULL|             NULL|cr56nez|          NULL|     2|       0|          buckus69| 6761|  1432889424|Then you got your...|         NULL|         0|               0|

In [3]:
dataset.dataframe.describe().show()

23/12/01 19:27:44 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+------------+--------+----------+--------------------+----------------------+--------------------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+------------------+-------------------+------------------+--------------------+
|summary|         created_utc|subreddit_id| link_id|      name|        score_hidden|author_flair_css_class|   author_flair_text|      id|removal_reason|              gilded|            archived|              author|             score|        retrieved_on|                body|     distinguished|             edited|  controversiality|           parent_id|
+-------+--------------------+------------+--------+----------+--------------------+----------------------+--------------------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+------------